In [31]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision

In [32]:
device = "cuda" if torch.cuda.is_available else "cpu"
device

'cuda'

In [33]:
torch.__version__

'2.4.1+cu124'

In [ ]:
from pathlib import Path
import requests
import zipfile

data_path = Path("data")
image_path = data_path / "pizza_steak_sushi"

if data_path.is_dir():
    print("Data sudah ada")
else:
    image_path.mkdir(exist_ok=True, parents=True)
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        print("Download data....")
        res = requests.get(
            "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
        )
        f.write(res.content)

    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Extract data...")
        zip_ref.extractall(path=image_path)

Download data....
